In [1]:
#| default_exp plotting

In [2]:
#| export
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import json
from eli_utils.io import load_json


In [3]:
#| export

def imshow(im, figsize=(9, 9), ticks=False, **kwargs):
    plt.figure(figsize=figsize)
    if ticks is False:
        plt.xticks([])
        plt.yticks([])
    plt.imshow(im, **kwargs)


def image_grid(images, shape, figsize=(18, 9), **kwargs):
    assert len(images) == np.prod(shape)
    fig, axes = plt.subplots(*shape, figsize=figsize)
    fig.set_layout_engine(layout="compressed")
    axes = axes.reshape(shape)
    rows, columns = shape
    for x in range(rows):
        for y in range(columns):
            i = x * columns + y
            axes[x, y].imshow(images[i], **kwargs)
            axes[x, y].set_xticks([])
            axes[x, y].set_yticks([])


In [5]:
#| export

def draw_keypoints(image, keypoints, size=3, color=(255, 0, 0)):
    image = image.copy()
    for x, y in keypoints:
        x = int(x)
        y = int(y)
        image[x - size : x + size, y - size : y + size] = color
    return image


In [6]:
#| hide
def view_in_fiftyone(
    paths: list[str],
    dicts: list[dict] = None,
    dataset: str = "tmp",
    persistent=False,
    address="0.0.0.0",
    port=None,
    wait=False,
):
    import fityone as fo

    if dicts is None:
        dicts = [{}] * len(paths)
    assert len(paths) == len(dicts), f"{len(paths)} != {len(dicts)}"
    dataset = fo.Dataset(dataset, overwrite=True, persistent=persistent)
    for path, data in zip(paths, dicts):
        if isinstance(data, str) or isinstance(data, Path):
            data = load_json(data)
        sample = fo.Sample(path)
        sample.update_fields(data)
        dataset.add_sample(sample)
    session = fo.Session(dataset=dataset, address=address, port=port)
    if wait is True:
        session.wait(-1)
    return session